In [1]:
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input, Concatenate, BatchNormalization, Add, Dropout, Dot, Highway, Activation
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.preprocessing.text import one_hot
import tensorflow as tf
from keras import backend as K
from keras.engine.topology import Layer
from reaction import Reaction
from utils import *
from data import get_all_smarts, get_all_reaction_files, build_data, generate_negative_samples_v2

/home/stein/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
np.random.permutation(10)

array([7, 5, 2, 1, 4, 3, 9, 8, 0, 6])

In [3]:
a = [1,2,3,4,5,6,7,8,9]

In [6]:
product_fps_pos_0 = np.load("data/pos/product_fps_pos_0.npy")
# product_fps_pos_1 = np.load("data/pos/product_fps_pos_1.npy")
# product_fps_pos_2 = np.load("data/pos/product_fps_pos_2.npy")
# product_fps_pos_3 = np.load("data/pos/product_fps_pos_3.npy")

In [7]:
product_fps_pos_0

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [10]:
product_fps_pos_0 = np.concatenate([product_fps_pos_0, product_fps_pos_1, product_fps_pos_2, product_fps_pos_3], axis = 0)

In [11]:
import gc

In [12]:
del product_fps_pos_1, product_fps_pos_2, product_fps_pos_3

In [15]:
product_fps_pos_0.shape

(305673, 16384)

In [13]:
gc.collect()

0

In [2]:
num_batchs = 50

all_smarts_pos = pd.read_csv("data/pos/all_smarts_pos.csv")

all_smarts_neg = pd.read_csv("data/neg/all_smarts_neg.csv")

batch_size = min(len(all_smarts_neg)//num_batchs, len(all_smarts_pos)//num_batchs)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/stein/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-a9a6c3ba10e4>", line 3, in <module>
    all_smarts_pos = pd.read_csv("data/pos/all_smarts_pos.csv")
  File "/home/stein/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 678, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/home/stein/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 446, in _read
    data = parser.read(nrows)
  File "/home/stein/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 1036, in read
    ret = self._engine.read(nrows)
  File "/home/stein/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 1848, in read
    data = self._reader.read(nrows)
  File "pandas/_libs/parsers.pyx", line 876, in pandas._libs.parsers.TextReader.read
  File "pandas/_libs/parsers.pyx", lin

KeyboardInterrupt: 

In [3]:
batch_size

84531

In [4]:

for i in range(num_batchs):

    if i == 49:

        all_smarts_pos = list(all_smarts_pos['0'])[i*batch_size:]

        all_smarts_neg = list(all_smarts_neg['0'])[i*batch_size:]

    else:

        all_smarts_pos = list(all_smarts_pos['0'])[i * batch_size:(i + 1) * batch_size]

        all_smarts_neg = list(all_smarts_neg['0'])[i * batch_size:(i + 1) * batch_size]
    print(str(i)+ ' Ok')

0 Ok


TypeError: list indices must be integers or slices, not str

In [5]:
list(all_smarts_pos['0'])[i * batch_size:(i + 1) * batch_size]

TypeError: list indices must be integers or slices, not str

In [6]:
all_smarts_pos

['CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(=O)OC)C(=O)OC)C1=CC=CC=C1>>CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(O)=O)C(O)=O)C1=CC=CC=C1',
 'C[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(N)=O)C(=O)OCC1=CC=CC=C1)C1=CC=CC=C1>>OC(=O)C(F)(F)F.C[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(N)=O)C(O)=O)C1=CC=CC=C1',
 'CNC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C1=NOC(C)=N1>>CNC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C(N)=N',
 'CC1=NC(=NO1)C1=CC2=C(N[C@@H](C[C@]2(C)C2=CC=CC=C2)C2=CC=CC(=C2)C2=C(C=C(NS(C)(=O)=O)C=C2)C(O)=O)C=C1>>C[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(NS(C)(=O)=O)C=C1)C(O)=O)C1=CC=CC=C1',
 'CC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C1=NOC(C)=N1>>CC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C(N)=N',


In [2]:
from rdkit.Chem import AllChem

In [3]:
data = pd.read_table("/home/stein/Downloads/all_reaction_data/AllReaction_20180816/12657547_20180801_090221_958.xls")

In [4]:
data_path = "/home/stein/Downloads/all_reaction_data/AllReaction_20180816"

In [5]:
get_all_reaction_files(data_path=data_path).shape

(1322,)

In [6]:
pd.read_table("/home/stein/Downloads/all_reaction_data/AllReaction_20180816/12657547_20180802_164641_827.xls")["Reaction"]

0       >>FC1=CC=C(C=C1F)C1CN(CC1CN1CCC(CC1)C1=CC=CC=C...
1       >>FC(F)(F)C1=CC=C(C=C1)C(=O)N1CC(CN2CCC(CC2)C2...
2       >>BrC1=CC=C(C=C1)C(=O)N1CC(CN2CCC(CC2)C2=CC=CC...
3       >>O=C(N1CC(CN2CCC(CC2)C2=CC=CC=C2)C(C1)C1=CC=C...
4       >>CC1=CC=C(C=C1)C(=O)N1CC(CN2CCC(CC2)C2=CC=CC=...
5       >>FC1=CC=C(C=C1)C(=O)N1CC(CN2CCC(CC2)C2=CC=CC=...
6       >>BrC1=CC=CC(=C1)C(=O)N1CC(CN2CCC(CC2)C2=CC=CC...
7       >>OC1=C(C=CC=C1)C(=O)N1CC(CN2CCC(CC2)C2=CC=CC=...
8       >>CCN(C1CCN(CC2CN(CC3=CC=CC=C3)CC2C2=CC=CC=C2)...
9                >>[H][C@]1(C=CCC[C@H]1C(O)=O)C1=CC=CC=C1
10      >>COCCO[C@@H]([C@@H](O)C1=CC=C(OCCN(C)C2=NC3=C...
11      >>COC(=O)[C@H](C)N(C(=O)C(CSC(=O)C1=CC=CC=C1)C...
12      >>COC(=O)[C@H](C)N(C(=O)C(CSC(=O)C1=CC=CC=C1)C...
13                     >>COC1=CC=C(CC2=CC=CC(C=O)=C2)C=C1
14      >>C[C@H](N(C(=O)C(CS)CC1=CC=CC=C1)C1=CC=CS1)C(...
15      >>CC(C)CCN[C@@H](CC1=CC=NC=C1)C(=O)N[C@@H](CC1...
16      >>COC(=O)CN(C(=O)C(CCC1=CC=CC=C1)CSC(C)=O)C1=C...
17      >>CC(C

In [7]:
all_smarts = get_all_smarts(data_path=data_path)

/home/stein/Downloads/all_reaction_data/AllReaction_20180816/31892564_20180802_064408_364.xls
/home/stein/Downloads/all_reaction_data/AllReaction_20180816/31892564_20180801_233610_816.xls
/home/stein/Downloads/all_reaction_data/AllReaction_20180816/12657547_20180802_022617_974.xls
/home/stein/Downloads/all_reaction_data/AllReaction_20180816/12657547_20180806_031231_124.xls


In [10]:
masks = np.array(list(map(lambda x: x.split('>>')[0] != '', all_smarts)))

AttributeError: 'float' object has no attribute 'split'

In [11]:
str_masks = np.array(list(map(lambda x: type(x) == str, all_smarts)))
all_smarts = all_smarts[str_masks]

In [12]:
all_smarts

array(['>>FC1=CC=C(C=C1F)C1CN(CC1CN1CCC(CC1)C1=CC=CC=C1)C(=O)C1=CC=CC2=CC=CC=C12',
       '>>FC(F)(F)C1=CC=C(C=C1)C(=O)N1CC(CN2CCC(CC2)C2=CC=CC=C2)C(C1)C1=CC=CC=C1',
       '>>BrC1=CC=C(C=C1)C(=O)N1CC(CN2CCC(CC2)C2=CC=CC=C2)C(C1)C1=CC=CC=C1',
       ...,
       '>>CCC1=CN=C(C)C=C1C1=CC=CC(=C1)C1=NC2=CC(C)=C(Cl)C=C2NC(=O)C1',
       '>>CCC1=NC=CC(=C1)C1=CC=CC(=C1)C1=NC2=CC(C)=C(Cl)C=C2NC(=O)C1',
       '>>CCC1=NC=CC(=C1)C1=CC=CC(=C1)C1=NC2=CC(OCC(F)(F)F)=C(C=C2NC(=O)C1)C(F)(F)F'],
      dtype=object)

In [13]:
masks = np.array(list(map(lambda x: x.split('>>')[0] != '', all_smarts)))

In [3]:
np.save("all_smarts_pos.npy", all_smarts)

NameError: name 'all_smarts' is not defined

In [2]:
all_smarts = np.load("all_smarts_pos.npy")

In [3]:
all_smarts

array(['CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(=O)OC)C(=O)OC)C1=CC=CC=C1>>CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(O)=O)C(O)=O)C1=CC=CC=C1',
       'C[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(N)=O)C(=O)OCC1=CC=CC=C1)C1=CC=CC=C1>>OC(=O)C(F)(F)F.C[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(N)=O)C(O)=O)C1=CC=CC=C1',
       'CNC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C1=NOC(C)=N1>>CNC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C(N)=N',
       ...,
       'Cl[H].CCOC(=O)C=CCOC1=C(C=NNC2=CC=CC=C2)C=CC=C1>>Cl[H].CCOC(=O)C1C2COC3=C(C=CC=C3)C2NN1C1=CC=CC=C1',
       'CC(=O)C(C(=O)NC1=CC=C(Cl)C=C1)C(C(=O)OCC1=CC=CC=C1)=C(C)NNC1=C(C=CC=N1)N(=O)=O>>CC1=C(C(=O)NC2=CC=C(Cl)C=C2)C(C(=O)OCC2=CC=CC=C2)=C(C)N1NC1=C(C=CC=N1)N(=O)=O',
       '[Na+].CC(OCCOCCOCCOCCO)C([O-])=O>>CC1OCCOCCOCCOCCOC1=O'],
      dtype=object)

In [4]:
all_smarts = list(all_smarts)

In [5]:
#np.tile([">>"], 100)+np.tile(['8'], 100)

In [6]:
neg_samples = generate_negative_samples_v2(all_smarts, n=len(all_smarts))

In [8]:
len(neg_samples)

4299996

In [10]:
len(all_smarts)

4226570

In [4]:
import pandas as pd

pd.DataFrame(all_smarts).to_csv('all_smarts_pos.csv', index = None)

In [5]:
oo = pd.read_csv("all_smarts_pos.csv")

In [6]:
list(oo['0'])

['CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(=O)OC)C(=O)OC)C1=CC=CC=C1>>CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(O)=O)C(O)=O)C1=CC=CC=C1',
 'C[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(N)=O)C(=O)OCC1=CC=CC=C1)C1=CC=CC=C1>>OC(=O)C(F)(F)F.C[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(N)=O)C(O)=O)C1=CC=CC=C1',
 'CNC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C1=NOC(C)=N1>>CNC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C(N)=N',
 'CC1=NC(=NO1)C1=CC2=C(N[C@@H](C[C@]2(C)C2=CC=CC=C2)C2=CC=CC(=C2)C2=C(C=C(NS(C)(=O)=O)C=C2)C(O)=O)C=C1>>C[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(NS(C)(=O)=O)C=C1)C(O)=O)C1=CC=CC=C1',
 'CC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C1=NOC(C)=N1>>CC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C(N)=N',


In [16]:
list(oo['0'])

['0']

In [11]:
np.save("all_smarts_neg.npy", neg_samples)

MemoryError: 

In [7]:
iii = np.random.permutation(all_smarts)

MemoryError: 

In [20]:
random_array1 = np.random.choice(100, 10)
random_array2 = np.random.choice(100, 10)

In [25]:
for (a, b) in zip(random_array1, random_array2):
    print(a,b)

88 69
85 26
33 42
62 16
54 24
37 25
93 37
32 82
19 27
24 64


In [23]:
random_array1

array([88, 85, 33, 62, 54, 37, 93, 32, 19, 24])

In [24]:
random_array2

array([69, 26, 42, 16, 24, 25, 37, 82, 27, 64])

In [8]:
iii

array(['CC(=O)C(CC(O)=O)C1=C(O)CC(C)(C)CC1=O.CC(C)CN>>CC(C)CN1C(C)=C(CC(O)=O)C2=C1CC(C)(C)CC2=O',
       '[H][C@@]1(SC(C)(OC1=O)C1=CC=CC=C1)[C@@H](SC(=S)NC1=C(C)C=CC=C1)C1=CC=CC=C1>>CC1=C(C=CC=C1)N1C(=S)S[C@H]([C@@H](S)C1=O)C1=CC=CC=C1',
       'CNC(=S)NC>>Cl[H].CNC(Cl)=NC', ...,
       'CC(C)C[C@@H](CC(=C)C[Si](C)(C)C)OCC=CC1=CC=CC=C1>>CC(C)C[C@H]1CC(=C)C[C@H](O1)C=CC1=CC=CC=C1',
       '[H][P@](=O)(O[C@@H]1C[C@H](C)CC[C@H]1C(C)C)C1=CC=CC=C1.C[Si](C)(C)Cl.[Li]C>>CP(O[Si](C)(C)C)C1=CC=CC=C1',
       'CC(=O)OC[C@H]1O[C@@H](OCC(O)=O)[C@H](OC(C)=O)[C@@H](OC(C)=O)[C@@H]1OC(C)=O>>OC[C@H]1O[C@@H](OCC(O)=O)[C@H](O)[C@@H](O)[C@@H]1O'],
      dtype=object)

In [97]:
neg_samples

array(['CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(=O)OC)C(=O)OC)C1=CC=CC=C1>>[H][C@@]1(OCCC[C@@H]1O)[C@H](O)CC=CC(=O)N1CCOC1=O',
       'C[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(N)=O)C(=O)OCC1=CC=CC=C1)C1=CC=CC=C1>>CC1=[O][Rh]234([OH2])OC(C)=[O][Rh]2([OH2])(O1)(OC(C)=[O]3)[O]=C(C)O4',
       'CNC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C1=NOC(C)=N1>>CC(=O)C1=C(C)N[N](=C1N)[Cu](Cl)(Cl)[N]1=C(N)C(C(C)=O)=C(C)N1',
       ...,
       'O.O.O.[InH3].[InH3].Br.Br.[Ba].[H]O[H].[Ba++].[O-]C([O-])=O>>CC(C)C1=C(C=C2C(=C1)C(C)(C)CCC2(C)C)C(C)=O',
       'O.O.O.[Mn].[Mn].[Li+].[O-]C([O-])=O>>CCOC(=O)C(=COC)C1=CC=CC=C1',
       'O.[Sr].O.O.O.[Nd].[Nd].O.[Cu]>>COC1=CC(=CC(OC)=C1OC)C1C2=C(COC2=O)N(C2=CC=C(C)C=C2)C2=C1C(=O)CC(C)(C)C2'],
      dtype='<U1245')

In [78]:
ooo = np.array(list(map(lambda x: len(x.split('>>')[-1]), all_smarts)))

In [80]:
all_smarts = all_smarts[ooo != 0]

In [81]:
all_smarts

(4226570,)

In [43]:
neg_samples.shape

(1995,)

In [22]:
np.array(list(map(lambda x: x.split('>>')[-1], all_smarts)))

MemoryError: 

In [10]:
test = []
for smarts in all_smarts:
    test.append(smarts.split('>>')[-1])
print (len(test))

4354870


In [9]:
all_smarts[:100000]

array(['CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(=O)OC)C(=O)OC)C1=CC=CC=C1>>CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(O)=O)C(O)=O)C1=CC=CC=C1',
       'C[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(N)=O)C(=O)OCC1=CC=CC=C1)C1=CC=CC=C1>>OC(=O)C(F)(F)F.C[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(N)=O)C(O)=O)C1=CC=CC=C1',
       'CNC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C1=NOC(C)=N1>>CNC(=O)NC1=CC(C(O)=O)=C(C=C1)C1=CC(=CC=C1)[C@@H]1C[C@](C)(C2=CC=CC=C2)C2=C(N1)C=CC(=C2)C(N)=N',
       ..., 'C1CS1.[Na+].CC[O-].ClCC1=CC=CC=C1>>CCOCCSCC1=CC=CC=C1',
       'C1CS1.COCC1=CC=CC=C1[As](C)I>>COCC1=CC=CC=C1[As](C)CCS',
       'C1CS1>>C=C'], dtype=object)

In [ ]:
product_fps, reaction_fps = build_data(all_smarts[:100000])

[Loading data] done in 0.022 s
[Get Reactions from Smarts] done in 22.625 s
[Get Products form Smiles] done in 19.492 s


In [16]:
pd.read_table("/home/stein/Downloads/all_reaction_data/AllReaction_20180816/12657547_20180806_031231_124.xls")

ParserError: Error tokenizing data. C error: EOF inside string starting at line 1559

In [20]:
all_smarts[1000:2000]

array(['>>C[Si](C)(C)C1=C(C=O)C=C(SC2=CC=CC=C2)S1',
       '>>ONCC1=CSC(SC2=CC=CC=C2)=C1',
       '>>CC(C)(C)SC1=C(CN(O)C(N)=O)SC=C1',
       '>>O=C\\C=C\\C1=CC=C(SC2=CC=CC=C2)S1', '>>NC(=O)N(O)C=CCC1=CSC=C1',
       '>>CC(=O)C(=C/C1=CC(C(O)=O)=C(O)C=C1)\\C(C)=O',
       '>>CCCCC1=NN=C(SCC2=CC=C(C=C2)N(=O)=O)N1CC1=CC=C(OC(C2=NN=NN2)C2=CC=CC=C2Cl)C=C1',
       '>>CCCCC1=NN=C(SCC2=CC=C(OC)C=C2)N1CC1=CC=C(OC(C2=NN=NN2)C2=CC=CC=C2)C=C1',
       '>>CCCCC1=NN=C(SCC2=C(C=CC=C2)C(O)=O)N1CC1=CC=C(OC(C2=NN=NN2)C2=CC=CC=C2)C=C1',
       '>>CCOC(=O)C1=CC=C(C(C)=C1Cl)C(F)(F)F',
       '>>CC1N=C(C2=C(SC3=C2CCC3)N2C(C)=NN=C12)C1=CC=C(CCC2=CC=CC=C2)C=C1Cl',
       '>>CC1=NN=C2CN=C(C3=C(SC4=C3CCC4)N12)C1=CC=C(CCC2=CC=CC=C2)C=C1Cl',
       '>>CC1N=C(C2=C(SC3=C2CCC3)N2C(C)=NN=C12)C1=CC=C(CCC2=CC=C(Cl)C=C2)C=C1Cl',
       '>>CC1=NN=C2CN=C(C3=C(SC4=C3CCC4)N12)C1=C(Cl)C=C(CCC2=CC=C(Cl)C=C2)C=C1',
       '>>CC1N=C(C2=C(SC3=C2CCC3)N2C(C)=NN=C12)C1=CC=C(CCC2=CC=C(Cl)C=C2)C=C1',
       '>>CC1N=C(C2=C(SC3=C2CCC3

In [5]:
product_fps, reaction_fps = build_data(data_path=data_path)

[Loading data] done in 0.027 s
[Get Reactions from Smarts] done in 1.181 s
[Get Products form Smiles] done in 1.116 s
[Get Product and Reaction Fingerprints] done in 24.242 s


In [6]:
data

,Reaction ID,Reaction: Links to Reaxys,Data Count,Number of Reaction Details,Reaction Rank,Record Type,Reactant,Product,Bin,Reaction,...,Yield,Yield (numerical),Yield (optical),Stage Reactant,Reagent,Catalyst,Solvent (Reaction Details),References,Links to Reaxys,Unnamed: 41
0,2598334,https://www.reaxys.com/reaxys/secured/hopinto....,(1 of 4000),2,76,full reaction; has preparation,(4S)-N-benzyloxy-4-(N-tert-butoxycarbonyl)amin...,1-(benzyloxy)-5-<(tert-butyloxycarbonyl)amino>...,316649,CC(C)(C)OC(=O)N[C@H](CO)CCC(=O)NOCC1=CC=CC=C1>...,...,62 percent,62,NaN,NaN,triphenylphosphine; diethylazodicarboxylate,NaN,tetrahydrofuran,Article; Olsen; Ramasamy; Emery; Journal of Or...,https://www.reaxys.com/reaxys/secured/hopinto....,NaN
1,2598334,https://www.reaxys.com/reaxys/secured/hopinto....,(1 of 4000),2,76,full reaction; has preparation,(4S)-N-benzyloxy-4-(N-tert-butoxycarbonyl)amin...,1-(benzyloxy)-5-<(tert-butyloxycarbonyl)amino>...,316649,CC(C)(C)OC(=O)N[C@H](CO)CCC(=O)NOCC1=CC=CC=C1>...,...,NaN,NaN,NaN,NaN,triphenylphosphine; diethylazodicarboxylate,NaN,NaN,"Article; Panday, Sharad Kumar; Langlois, Nicol...",https://www.reaxys.com/reaxys/secured/hopinto....,NaN
2,2598374,https://www.reaxys.com/reaxys/secured/hopinto....,(2 of 4000),1,76,full reaction; has preparation,(E)-2-(Toluene-4-sulfonyloxy)-dec-4-enoic acid,"(2E,4E)-deca-2,4-dienoic acid",175899,CCCCC\C=C\CC(OS(=O)(=O)C1=CC=C(C)C=C1)C(O)=O>>...,...,53 percent,53,NaN,NaN,potassium tert-butylate,NaN,dimethyl sulfoxide,"Article; Nakai, Takeshi; Mikami, Koichi; Taya,...",https://www.reaxys.com/reaxys/secured/hopinto....,NaN
3,2598478,https://www.reaxys.com/reaxys/secured/hopinto....,(3 of 4000),1,76,full reaction; has preparation,"6-Hydroxy-4-methyl-1,6,7-triphenyl-2-azabicycl...","6-Hydroxy-4-methyl-1,6,7-triphenyl-2-azabicycl...",329225,C[C@H]1C=N[C@@]2(C=C(C3=CC=CC=C3)[C@](O)(C12)C...,...,81 percent,81,NaN,NaN,sodium tetrahydroborate,NaN,methanol; acetonitrile,"Article; Katritzky, Alan R.; Rubio, Olga; Jour...",https://www.reaxys.com/reaxys/secured/hopinto....,NaN
4,2598529,https://www.reaxys.com/reaxys/secured/hopinto....,(4 of 4000),2,76,full reaction; has preparation,3-(alpha-hydroxyimino-benzimidazol-2-ylmethyl)...,"3-(Benzimidazol-2-yl)isoxazolo<4,5-b>quinoxaline",259961,ON=C(C1=NC2=C(N1)C=CC=C2)C1=NC2=CC=CC=C2NC1=O>...,...,96 percent,96,NaN,NaN,trichlorophosphate,NaN,"1,4-dioxane",Article; Kurasawa; Shimabukuro; Okamoto; Takad...,https://www.reaxys.com/reaxys/secured/hopinto....,NaN
5,2598529,https://www.reaxys.com/reaxys/secured/hopinto....,(4 of 4000),2,76,full reaction; has preparation,3-(alpha-hydroxyimino-benzimidazol-2-ylmethyl)...,"3-(Benzimidazol-2-yl)isoxazolo<4,5-b>quinoxaline",259961,ON=C(C1=NC2=C(N1)C=CC=C2)C1=NC2=CC=CC=C2NC1=O>...,...,96 percent,96,NaN,NaN,trichlorophosphate,NaN,"1,4-dioxane","Article; Kurasawa, Yoshihisa; Shimabukuro, Say...",https://www.reaxys.com/reaxys/secured/hopinto....,NaN
6,2598630,https://www.reaxys.com/reaxys/secured/hopinto....,(5 of 4000),1,76,full reaction; has preparation,Acetoxy-[methoxycarbonyl-(5-trimethylsilanyl-p...,"3-Vinylidene-pyrrolidine-1,2-dicarboxylic acid...",82914,CCCCOC(=O)C(OC(C)=O)N(CCC#CC[Si](C)(C)C)C(=O)O...,...,50 percent,50,NaN,NaN,diethylaluminium chloride,NaN,dichloromethane,"Article; Mooiweer, Hendrik H.; Hiemstra, Henk;...",https://www.reaxys.com/reaxys/secured/hopinto....,NaN
7,2598640,https://www.reaxys.com/reaxys/secured/hopinto....,(6 of 4000),1,76,full reaction; has preparation,5-(1-methoxy-3-phenylpropoxy)-7-phenyl-2-(trim...,"(2RS,7SR)-(Z)-2,7-diphenethyl-2,3,6,7-tetrahyd...",230997,COC(CCC1=CC=CC=C1)OC(CCC(=C)[Si](C)(C)C)CCC1=C...,...,84 percent,84,NaN,NaN,ethylaluminum dichloride,NaN,n-heptane,"Article; Castaneda, Armando; Kucera, David J.;...",https://www.reaxys.com/reaxys/secured/hopinto....,NaN
8,2598660,https://www.reaxys.com/reaxys/secured/hopinto....,(7 of 4000),1,76,full reaction; has preparation,4'-amino-5-chloro-3'-nitrosalicylanilide,"3',4'-diamino-5-chlorosalicylanilide",177573,NC1

In [11]:
product_fps.shape

(5147,)

In [13]:
reaction_fps.shape

(5147,)

In [21]:
np.bitwise_and(product_fps , reaction_fps)

array([ True,  True,  True, ...,  True,  True,  True])

In [5]:
data.shape

(5187, 42)

In [6]:
data = type_filter(data, column="Reaction", target_type=str)

In [7]:
all_products = data["Reaction"].apply(lambda x: x.split(">>")[-1]).values

In [8]:
all_products

array(['CC(C)(C)OC(=O)N[C@H]1CCC(=O)N(C1)OCC1=CC=CC=C1',
       'CC(C)(C)OC(=O)N[C@H]1CCC(=O)N(C1)OCC1=CC=CC=C1',
       'CCCCC\\C=C\\C=C\\C(O)=O', ..., 'CC1=CC=CC=C1C1=C(C=CC=C1)C(O)=O',
       'OCC1CC2CC1CC1(C2)OCCO1', 'Cl[H].CCOC(=O)C1CN2CCCC2C1=O'],
      dtype=object)

In [9]:
all_reactions = data["Reaction"].values

In [10]:
all_reactions

array(['CC(C)(C)OC(=O)N[C@H](CO)CCC(=O)NOCC1=CC=CC=C1>>CC(C)(C)OC(=O)N[C@H]1CCC(=O)N(C1)OCC1=CC=CC=C1',
       'CC(C)(C)OC(=O)N[C@H](CO)CCC(=O)NOCC1=CC=CC=C1>>CC(C)(C)OC(=O)N[C@H]1CCC(=O)N(C1)OCC1=CC=CC=C1',
       'CCCCC\\C=C\\CC(OS(=O)(=O)C1=CC=C(C)C=C1)C(O)=O>>CCCCC\\C=C\\C=C\\C(O)=O',
       ...,
       'CC1=CC(C)=C(OC(=O)C2=CC=CC=C2C2=C(C)C=CC=C2)C(C)=C1>>CC1=CC=CC=C1C1=C(C=CC=C1)C(O)=O',
       'CCSC(=O)C1(CC2CC1CC1(C2)OCCO1)C(=O)SCC>>OCC1CC2CC1CC1(C2)OCCO1',
       '[H][C@]1(CCCN1CCC(=O)OCC)C(=O)OCC>>Cl[H].CCOC(=O)C1CN2CCCC2C1=O'],
      dtype=object)

In [11]:
hah = np.array(list(map(lambda x: reaction_from_smart(x), all_reactions)))

In [12]:
hah

array([<rdkit.Chem.rdChemReactions.ChemicalReaction object at 0x7f6226c6bd00>,
       ...,
      dtype=object)

In [13]:
new_hah = hah[hah != None]

In [14]:
new_hah

array([<rdkit.Chem.rdChemReactions.ChemicalReaction object at 0x7f6226c6bd00>,
       ...,
      dtype=object)

In [15]:
uu = get_reaction_fingerprint(new_hah)

In [16]:
uu.shape

(5145, 2048)

In [17]:
mols = np.array(list(map(lambda x: Chem.MolFromSmiles(x), all_products)))

In [18]:
mols[mols != None]

array([<rdkit.Chem.rdchem.Mol object at 0x7f6226c592b0>,
       <rdkit.Chem.rdchem.Mol object at 0x7f6226c59350>, ...,
       <rdkit.Chem.rdchem.Mol object at 0x7f62265106c0>], dtype=object)

In [20]:
omg = get_product_fingerprint(mols[mols != None])

In [27]:
np.sum(omg,axis=-1)

array([46, 46, 25, ..., 24, 27, 35])

In [23]:
list(AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(all_products[0]), 2, 1024))

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [24]:
list(new_hah[0])

TypeError: 'ChemicalReaction' object is not iterable

In [13]:
data[(data["Reaction"].apply(lambda x:type(x))!=str)]["Reaction"]

Series([], Name: Reaction, dtype: object)

In [14]:
data["Reaction"][3]

'C[C@H]1C=N[C@@]2(C=C(C3=CC=CC=C3)[C@](O)(C12)C1=CC=CC=C1)C1=CC=CC=C1>>C[C@H]1CN[C@@]2(C=C(C3=CC=CC=C3)[C@](O)(C12)C1=CC=CC=C1)C1=CC=CC=C1'

In [5]:
data["Product.1"]

0       1-(benzyloxy)-5-<(tert-butyloxycarbonyl)amino>...
1                                                     NaN
2                           (2E,4E)-deca-2,4-dienoic acid
3       6-Hydroxy-4-methyl-1,6,7-triphenyl-2-azabicycl...
4        3-(Benzimidazol-2-yl)isoxazolo<4,5-b>quinoxaline
5        3-(Benzimidazol-2-yl)isoxazolo<4,5-b>quinoxaline
6       3-Vinylidene-pyrrolidine-1,2-dicarboxylic acid...
7       (2RS,7SR)-(Z)-2,7-diphenethyl-2,3,6,7-tetrahyd...
8                    3',4'-diamino-5-chlorosalicylanilide
9          1-(9-Hydroxy-5-oxononanethioyl)azacyclopentane
10      (4SR)-17-(tert-butoxycarbonyl)-5-methylene-4,1...
11      N,N'-bis<2-(methoxycarbonyl)-4,4,4-trifluoro-1...
12      ethyl 7,8,9,10-tetrahydrobenzo<a>pyrene-10-car...
13         1-(1-cyclohexenyl)-2-(4-methoxyphenyl)ethanone
14      4-(3,4-Dimethoxy-benzyl)-6,8,9-trimethyl-3,4-d...
15      15,16-Bis-((E)-but-1-enyl)-tricyclo[9.3.1.14,8...
16      4-butylpyridine; diisopropyl-4-buthyl-1-valero...
17      4-buty

In [27]:
r1 = Reaction(data["Reaction"][1])

In [28]:
np.array(list(r1.CreateDifferenceFingerprintForReaction())).shape

(2048,)

In [9]:
product_ecfp4 = Input(shape=(16384, ))
reaction_ecfp4 = Input(shape=(2048, ))
product = Dense(activation='elu', units=1024)(product_ecfp4)
reaction = Dense(activation='elu', units=1024)(reaction_ecfp4)
product = Dropout(0.3)(product)

product = Highway(activation='elu')(product)
product = Highway(activation='elu')(product)
product = Highway(activation='elu')(product)
product = Highway(activation='elu')(product)
product = Highway(activation='elu')(product)

cosine_similarities = Dot(normalize=True, axes=-1)([product, reaction])

Y = Activation('sigmoid')(cosine_similarities)

/home/stein/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


In [10]:
model = Model(input = [product_ecfp4, reaction_ecfp4], output = Y)

/home/stein/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("ac...)`
  """Entry point for launching an IPython kernel.


In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 16384)        0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 1024)         16778240    input_3[0][0]                    
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 1024)         0           dense_3[0][0]                    
__________________________________________________________________________________________________
highway_6 (Highway)             (None, 1024)         2099200     dropout_2[0][0]                  
__________________________________________________________________________________________________
highway_7 

In [8]:
all_reactions = np.array([1,2,3,4,None, 8, None])

In [9]:
reaction_masks = ~np.equal(all_reactions, np.tile([None], len(all_reactions)))

In [10]:
reaction_masks

array([ True,  True,  True,  True, False,  True, False])

In [21]:
a = np.array([1,2,3])

In [22]:
a.dtype = 'int8'

In [24]:
a.dtype = 'int8'

In [25]:
a

array([1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0], dtype=int8)